In [2]:
import numpy as np
import cv2
import os
from sklearn.utils import shuffle

In [3]:
xtrain= []
ytrain = []
xtest = []
ytest = []
xval =[]
yval = []
for dir in ['Train', 'Test', 'Validation']:
	for label in os.listdir(dir):
		for pic in os.listdir(dir + '/' + label):
			img = cv2.imread(dir + '/' + label + '/' + pic)
			img = cv2.resize(img, dsize=(300,300), interpolation=cv2.INTER_CUBIC)
			img = img / 255
			if dir == 'Train':
				xtrain.append(img)
				if label == 'Mask':
					ytrain.append(1)
				else:
					ytrain.append(0)
			elif dir == 'Test':
				xtest.append(img)
				if label == 'Mask':
					ytest.append(1)
				else:
					ytest.append(0)
			elif dir == 'Validation':
				xval.append(img)
				if label == 'Mask':
					yval.append(1)
				else:
					yval.append(0)

In [5]:
xtrain = np.array(xtrain)
ytrain = np.array(ytrain)
xtest = np.array(xtest)
ytest = np.array(ytest)
xval = np.array(xval)
yval = np.array(yval)

print(f'Number of training samples : {len(xtrain)}')
print(f'xtrain shape : {xtrain.shape}')
print(f'ytrain shape : {ytrain.shape}')

print(f'Number of testing samples : {len(xtest)}')
print(f'xtest shape : {xtest.shape}')
print(f'ytest shape : {ytest.shape}')

print(f'Number of validation samples : {len(xval)}')
print(f'xval shape : {xval.shape}')
print(f'yval shape : {yval.shape}')

Number of training samples : 600
xtrain shape : (600, 300, 300, 3)
ytrain shape : (600,)
Number of testing samples : 100
xtest shape : (100, 300, 300, 3)
ytest shape : (100,)
Number of validation samples : 306
xval shape : (306, 300, 300, 3)
yval shape : (306,)


In [7]:
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dropout,Dense
from keras.models import Sequential,load_model
from keras.callbacks import ModelCheckpoint
model=Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(300,300,3)))
model.add(MaxPooling2D() )
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D() )
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D() )
model.add(Flatten())
model.add(Dense(100,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
checkpoint = ModelCheckpoint('FaceMask.h5', save_best_only=True)
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 298, 298, 32)      896       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 149, 149, 32)     0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 147, 147, 32)      9248      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 73, 73, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 71, 71, 32)        9248      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 35, 35, 32)      

In [8]:
model.fit(xtrain, ytrain, epochs=10, validation_data=(xval, yval), verbose=1, callbacks=[checkpoint])

Epoch 1/10
19/19 [==============================] - 37s 2s/step - loss: 0.5285 - accuracy: 0.7467 - val_loss: 0.4063 - val_accuracy: 0.8268
Epoch 2/10
19/19 [==============================] - 33s 2s/step - loss: 0.3249 - accuracy: 0.8600 - val_loss: 0.2788 - val_accuracy: 0.9118
Epoch 3/10
19/19 [==============================] - 32s 2s/step - loss: 0.2313 - accuracy: 0.9167 - val_loss: 0.2645 - val_accuracy: 0.8954
Epoch 4/10
19/19 [==============================] - 31s 2s/step - loss: 0.1327 - accuracy: 0.9517 - val_loss: 0.3324 - val_accuracy: 0.8954
Epoch 5/10
19/19 [==============================] - 31s 2s/step - loss: 0.1211 - accuracy: 0.9483 - val_loss: 0.2439 - val_accuracy: 0.9085
Epoch 6/10
19/19 [==============================] - 31s 2s/step - loss: 0.0833 - accuracy: 0.9700 - val_loss: 0.3303 - val_accuracy: 0.9052
Epoch 7/10
19/19 [==============================] - 35s 2s/step - loss: 0.0504 - accuracy: 0.9800 - val_loss: 0.3819 - val_accuracy: 0.8954
Epoch 8/10
19/19 [==

In [9]:
from sklearn.metrics import confusion_matrix

In [10]:
trainPred = model.predict(xtrain)
trainPred = np.round(trainPred)
confusion_matrix(ytrain, trainPred)

array([[300,   0],
       [  0, 300]], dtype=int64)

In [11]:
testPred = model.predict(xtest)
testPred = np.round(testPred)
confusion_matrix(ytest, testPred)

array([[49,  1],
       [ 9, 41]], dtype=int64)

In [12]:
valPred = model.predict(xval)
valPred = np.round(valPred)
confusion_matrix(yval, valPred)

array([[150,   3],
       [ 28, 125]], dtype=int64)

In [ ]:
model.save('faceMaskDetection.h5')

In [ ]:
from keras.models import load_model
model = load_model('faceMaskDetection.h5')

In [13]:
inp = np.array([xtrain[5]])
print(np.round(model.predict(inp)))
print(ytrain[0])

[[1.]]
1
